# Gym Exercise Recommendation

The idea of this project is to create a recommendation based algorithm built on top of a gym exercise dataset.
The goal for v1 release is to have a model that will recommend exercises based on user level (beginner, intermediate, advanced), type of exercise/training, equipment available and body part.
For the v2 release, I'd like to have some user information/profiles in a way that a content-based and collaborative filtering.
For the v3 release, I'm aiming to have a frontend webapp/mobile app users can interact with.
For the v4 release, the goal is to also include meal planning. Users will be suggested meals based on their weight, age, body-type and gym goal (getting more swollen, losing weight, etc).

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandasql import sqldf
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import preprocessing 
from sklearn.preprocessing import LabelEncoder
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity

In [49]:
# Reads CSV
data = pd.read_csv(r'/Users/steelo/Library/CloudStorage/OneDrive-Personal/Dev/codeProjects/gym_recommendation/megaGymDataset.csv') 

In [50]:
data.rename( columns={'Unnamed: 0':'ID'}, inplace=True )
data

,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0,NaN
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2913,2913,EZ-bar skullcrusher-,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,8.1,Average
2914,2914,Lying Close-Grip Barbell Triceps Press To Chin,NaN,Strength,Triceps,E-Z Curl Bar,Beginner,8.1,Average
2915,2915,EZ-Bar Skullcrusher - Gethin Variation,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,NaN,NaN
2916,2916,TBS Skullcrusher,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,NaN,NaN


In [51]:
sqldf("SELECT * FROM data GROUP BY Type", globals())

,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc
0,149,Roc Everest,None,Cardio,Abdominals,Other,Intermediate,NaN,None
1,1217,Kneeling Jump Squat,None,Olympic Weightlifting,Glutes,Barbell,Beginner,7.6,Average
2,96,Sledgehammer swing,The sledgehammer swing is an exercise working ...,Plyometrics,Abdominals,Other,Intermediate,8.6,Average
3,917,Bench Press With Short Bands,None,Powerlifting,Chest,Bands,Beginner,0.0,None
4,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0,None
5,204,Stomach Vacuum,None,Stretching,Abdominals,Body Only,Beginner,8.5,Average
6,1159,Forward Drag with Press,None,Strongman,Chest,Other,Beginner,6.0,Average


In [52]:
le = LabelEncoder() # Create a LabelEncoder object

# Fit the LabelEncoder to the data and transform it
data['Type_encoded'] = le.fit_transform(data['Type'])
data['Equipment_encoded'] = le.fit_transform(data['Equipment'])
data['BodyPart_encoded'] = le.fit_transform(data['BodyPart'])

# One-hot encode the 'Level' column
data = pd.concat([data, pd.get_dummies(data['Level'], prefix='Level')], axis=1)

data

,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc,Type_encoded,Equipment_encoded,BodyPart_encoded,Level_Beginner,Level_Expert,Level_Intermediate
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0,NaN,4,0,0,False,False,True
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,NaN,NaN,4,0,0,False,False,True
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,NaN,NaN,4,0,0,False,False,True
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,NaN,NaN,4,0,0,False,False,True
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,NaN,NaN,4,0,0,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913,2913,EZ-bar skullcrusher-,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,8.1,Average,4,5,16,False,False,True
2914,2914,Lying Close-Grip Barbell Triceps Press To Chin,NaN,Strength,Triceps,E-Z Curl Bar,Beginner,8.1,Average,4,5,16,True,False,False
2915,2915,EZ-Bar Skullcrusher - Gethin Variation,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,NaN,NaN,4,5,16,False,False,True
2916,2916,TBS Skullcrusher,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,NaN,NaN,4,5,16,False,False,True


In [ ]:
heatmap_columns = ['Type_encoded', 'Equipment_encoded', 'BodyPart_encoded', 'Level_Beginner', 'Level_Intermediate', 'Level_Expert']
sns.heatmap(data[heatmap_columns].corr(), annot=True, cmap='coolwarm')


In [ ]:
# Compute Pearson correlation coefficient and p-value
columns_for_corr = ['Type_encoded', 'Equipment_encoded', 'BodyPart_encoded', 'Level_Beginner', 'Level_Intermediate', 'Level_Expert']
def calc_correlation():
    for i in range(len(columns_for_corr)):
        for j in range(i + 1, len(columns_for_corr)):
            correlation_coefficient, p_value = pearsonr(data[columns_for_corr[i]], data[columns_for_corr[j]])
            print(f"Pearson correlation coefficient for '{columns_for_corr[i]}' and '{columns_for_corr[j]}':", correlation_coefficient)
            print("P-value:", p_value)
calc_correlation()

In [ ]:
columns = ['Type_encoded', 'Equipment_encoded', 'BodyPart_encoded', 'Level_Beginner', 'Level_Intermediate', 'Level_Expert'] # Create a new DF
data1 = data[columns]

similarity_matrix = cosine_similarity(data1) # Calculate the Cosine Similarity between each pair of exercises

similarity_df = pd.DataFrame(similarity_matrix, index=data1.index, columns=data1.index) # Convert the similarity matrix to a DataFrame for better readability

similarity_df.index = data['ID'] # Replace the indices with the exercise titles

similarity_df.columns = data['ID'] # Replace the column names with the exercise titles

similarity_df

In [ ]:
def recommend_exercises(ID, similarity_df, num_recommendations=5):
    exercise_similarities = similarity_df[ID]     # Get the column corresponding to the given exercise ID
    similar_exercises = exercise_similarities.sort_values(ascending=False)     # Sort the exercises in descending order of similarity
    top_exercises = similar_exercises.head(num_recommendations + 1)   # Get the top N exercises
    top_exercises = top_exercises.loc[top_exercises.index != ID]    # Remove the given exercise from the recommendations

    # Create a DataFrame with the exercise IDs and their corresponding titles
    top_exercises_df = pd.DataFrame(top_exercises)
    top_exercises_df['Title'] = top_exercises_df.index.map(data.set_index('ID')['Title'])
    top_exercises_df.reset_index(inplace=True)
    top_exercises_df.columns = ['ID', 'Similarity', 'Title']

    return top_exercises_df

# Test the function with a valid exercise ID from your DataFrame
valid_ID = data['ID'].iloc[0]  # Replace this with a valid ID from your DataFrame
recommendations = recommend_exercises(valid_ID, similarity_df)
print(recommendations)

In [ ]:
sqldf("SELECT * FROM data WHERE Type = 'Strength' AND BodyPart = 'Chest' AND Equipment = 'Barbell'", globals())
# sqldf("SELECT * FROM data WHERE ID IN ('2', '3', '4', '5', '6')", globals())

In [ ]:
user_inputs = []

def get_user_input(prompt):
    user_input = input(prompt)
    user_inputs.append(user_input)
    return user_input

def recommend_based_on_user_input(data, similarity_df, num_recommendations=5):
    # Ask the user for their preferences
    type_preference = get_user_input("What type of exercise would you like to do? ")
    body_part_preference = get_user_input("What body part would you like to train? ")
    equipment_preference = get_user_input("What equipment do you have available? ")

    # Encode the user's preferences
    type_encoded = le.fit_transform([type_preference])
    body_part_encoded = le.fit_transform([body_part_preference])

    # Create a DataFrame that contains the user's preferences
    user_preferences = pd.DataFrame({
        'Type_encoded': type_encoded,
        'BodyPart_encoded': body_part_encoded,
    }, index=[0])

    # If the user inputs 'any' for the equipment, skip the equipment preference
    if equipment_preference.lower() != 'any':
        equipment_encoded = le.fit_transform([equipment_preference])
        user_preferences['Equipment_encoded'] = equipment_encoded
        data = data[data['Equipment_encoded'] == equipment_encoded[0]]

    # Filter the data to only include exercises that match the user's body part preference
    data_filtered = data[data['BodyPart_encoded'] == body_part_encoded[0]].copy()

    # Compute the cosine similarity between the user's preferences and the filtered data
    similarity_scores = cosine_similarity(data_filtered[user_preferences.columns], user_preferences)

    # Sort the exercises by their similarity to the user's preferences
    data_filtered['Similarity'] = similarity_scores
    recommendations = data_filtered.sort_values(by='Similarity', ascending=False).head(num_recommendations)

    # Return the recommended exercises
    return recommendations[['ID', 'Title', 'Similarity']]

recommendations = recommend_based_on_user_input(data, similarity_df)
print(f"User inputs were: {user_inputs}.")
print(f"The recommended exercises are the following: {recommendations}.")

In [ ]:
sqldf("SELECT * FROM data WHERE ID IN ('0', '1', '2', '3', '4')", globals())

Need to fix the recommendation algorithm. It's not working as expected. 
The column Level needs to be one-hot encoded instead of being label encoded.
It is returning exercises that don't have the equipment the user is giving as an input.

## Data visualization

In [ ]:
sqldf("SELECT TItle, Type, BodyPart, Equipment FROM data GROUP BY Type, BodyPart, Equipment", globals())